In [ ]:
import os
from dotenv import load_dotenv
from google.genai import types
from google.adk.agents import Agent, ParallelAgent, SequentialAgent
from google.adk.models.google_llm import Gemini
from pathlib import Path
from agents_intensive_capstone.agents import blue_hat_agent, green_hat_agent, yellow_hat_agent

## Configure Retry Options¶

In [ ]:
load_dotenv()

# 2. Access the variable
api_key = os.getenv("GOOGLE_API_KEY")
print("✅ Gemini API key setup complete.")

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [ ]:
model = Gemini(
    model="gemini-2.5-flash-lite",
    retry_options=retry_config,
)

In [ ]:
blue_hat = blue_hat_agent.BlueHatAgent.create(
    model=model,
    prompt_folder=Path("../src/agents_intensive_capstone/prompts"),
)


In [ ]:
# GREEN HAT: Creativity & Alternatives
green_hat = green_hat_agent.GreenHatAgent(
    model=model,
    prompt_folder=Path("../src/agents_intensive_capstone/prompts"),
)

In [ ]:
# YELLOW HAT: Optimism & Benefits
yellow_hat = yellow_hat_agent.YellowHatAgent(
    model=model,
    prompt_folder=Path("../src/agents_intensive_capstone/prompts"),
)

## Six thinking hats

In [ ]:
# Define the 5 "Thinking" Agents (The Team)
# WHITE HAT: Facts & Data
white_hat = Agent(
    name="WhiteHat",
    model=model,
    instruction="You are the White Hat. Focus ONLY on available data, facts, and information gaps. Be objective and neutral. Do not offer opinions, only verifiable facts or questions about missing data.",
)

# RED HAT: Emotions & Intuition
red_hat = Agent(
    name="RedHat",
    model=model,
    instruction="You are the Red Hat. Focus on intuition, hunches, and emotional reaction. How does this problem make users or the team feel? You do not need to justify your feelings with logic.",
)

# BLACK HAT: Caution & Risk
black_hat = Agent(
    name="BlackHat",
    model=model,
    instruction="You are the Black Hat. Play the devil's advocate. Identify specific risks, potential failure points, downsides, and why this idea might NOT work. Be critical.",
)

In [ ]:
# Create the Parallel Group
# This runs all 5 agents at the same time on the user prompt
thinking_team = ParallelAgent(
    name="SixHatsBrainstorm",
    sub_agents=[white_hat, red_hat, black_hat, yellow_hat, green_hat]
)

In [ ]:
# Create the Final Workflow
# First run the team (Parallel), then run the manager (Sequential)
solver_workflow = SequentialAgent(
    name="SixHatsSolver",
    sub_agents=[thinking_team, blue_hat]
)

In [ ]:
from google.adk.runners import InMemoryRunner
from google.adk.plugins.logging_plugin import (
    LoggingPlugin,
)  # <---- 1. Import the Plugin
from google.genai import types
import asyncio

runner = InMemoryRunner(
    agent=solver_workflow,
    plugins=[
        LoggingPlugin()
    ], 
)
print("✅ Runner configured")

In [ ]:
response = await runner.run_debug("Should we switch our backend database from PostgreSQL to a NoSQL solution for our startup?")